# Create all incorrect forumla for SVAMP

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import math
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

In [4]:
number_list = ['number0', 'number1', 'number2', 'number3', 'number4', 'number5', 'number6']
operation_dict = {'+': 'add', '-':'substract', '/': 'divide', '*': 'multiply'}
operation_list = ['*', '+', '-', '/']
operation_list2 = ["multiply", "add", "substract", "divide"]

def tree2linear(equation, operation_dict = operation_dict, number_list = number_list, operation_list = operation_list):
    """
    - + number1 number2 number3 --> add(number1, number2) | subtract(#0, number3) 
    - number1 + number2 number3 --> add(number2, number3) | subtract(#0, number1) 
    """
    #equation = "- number1 + number2 number3"
    #equation = "* - 1.0 * + 1.0 * number0 0.01 - 1.0 * number0 0.01 100.0"
    operations, numbers = _occurance(operation_list, number_list, equation)
    o_index, v_index, c_index =  get_index(operation_list, number_list, equation)
    list_equation = equation.split(' ')
    if len(operations) ==1:
        i = 0
        if c_index==[]: 
            linear_equation = '#'+str(0) +": " + operation_dict[operations[0]]+' ( '+numbers[i] +', '+numbers[i+1]+' )' +' | '
        else: 
            linear_equation = '#'+str(0) +": " + operation_dict[operations[0]]+' ( '+list_equation[i+1] +', '+list_equation[i+2]+' )' +' | '
    
    elif len(operations)==2:
        #if the pattern is operation2 operation1 numberX numberY numberZ
        operations = operations[::-1]
        if 0 in o_index and 1 in o_index:
            for i in range(len(operations)):
                if i == 0:
                    linear_equation = '#'+str(i) +": " + operation_dict[operations[i]]+' ( '+list_equation[i+2] +', '+list_equation[i+3]+' )' +' | '
                else:
                    linear_equation = linear_equation +'#'+str(i)+ ": " + operation_dict[operations[i]]+' ( '+ '#'+str(i-1) +', '+list_equation[i+3]+' )' +' | '
        else: 
        # if the pattern is operation2 numberZ operation1 numberX numberY
            start_index = len(operations)
            for i in range(len(operations)):
                if i == 0:
                    linear_equation = '#'+str(i) +": " + operation_dict[operations[i]]+' ( '+list_equation[start_index+i+1] +', '+list_equation[start_index+i+2]+' )' +' | '
                else:
                    linear_equation = linear_equation +'#'+str(i)+ ": " + operation_dict[operations[i]]+' ( '+list_equation[start_index-i]+ ', #'+str(i-1) + ' )' +' | '
                    
    elif len(operations) >2:
        operations = operations[::-1]
        start_index = len(operations)
        if not o_index[0] and all(y-x==1 for x, y in zip(o_index, o_index[1:])): # check if the o_index is incrementing one at a time 0,1,2,3,..., len(operations)
            for i in range(len(operations)):
                if i == 0:
                    linear_equation = '#'+str(i) +": " + operation_dict[operations[i]]+' ( '+ list_equation[start_index+i] +', '+list_equation[start_index+i+1]+' )' +' | '
                else:
                    linear_equation = linear_equation +'#'+str(i)+ ": " + operation_dict[operations[i]]+' ( '+ '#'+str(i-1) +', '+list_equation[start_index +i + 1]+' )' +' | '
                    
        else:
            linear_equation = equation
    else: 
        linear_equation = equation

    return linear_equation


def _occurance(operation_list, number_list, equation): 
    operations = []
    numbers = []
    list_equation = equation.split(' ')
    
    for i in range(len(list_equation)):
        if list_equation[i] in operation_list:
            operations.append(list_equation[i])
        elif list_equation[i] in number_list:
            numbers.append(list_equation[i])
    
    return operations, numbers

def get_index(operation_list, number_list, equation): 
    operation_index = []
    value_index = []
    constant_index = []
    list_equation = equation.split(' ')
    for i in range(len(list_equation)): 
        if list_equation[i] in operation_list: 
            operation_index.append(i)
        elif list_equation[i] in number_list: 
            value_index.append(i)
        else: 
            constant_index.append(i)

    return operation_index, value_index, constant_index

In [26]:
def num(list):
    s = ""
    for l in list:
        s = s + str(l) + " "
    return s

In [27]:
# open data
file = "SVAMP"
if file == "SVAMP":       
    SVAMP = pd.read_json(file + ".json")
    SVAMP["Numbers"] = SVAMP["Numbers"].apply(num)
    question_label = "Question"
else:
    SVAMP = pd.read_csv(file + ".csv")
    question_label = "Ques"
    SVAMP["Linear_Formula"] = SVAMP['Equation'].transform(tree2linear)
    SVAMP["Linear_Formula"].head()
print(SVAMP.shape)
SVAMP.drop(SVAMP[SVAMP['Equation'] == SVAMP['Linear_Formula']].index, inplace=True)
print(SVAMP.shape)
SVAMP.head()

(999, 8)
(999, 8)


,Answer,Body,Equation,ID,Linear_Formula,Numbers,Question,Type
0,51,Each pack of dvds costs number0 dollars. If th...,( 76.0 - 25.0 ),chal-1,"#0: substract ( number0, number1 ) | EOS",76.0 25.0,How much do you have to pay to buy each pack?,Subtraction
1,1,Dan had $ number0 left with him after he bough...,( 4.0 - 3.0 ),chal-2,"#0: substract ( number1, number0 ) | EOS",4.0 3.0,How much did the candy bar cost?,Subtraction
2,17,Paco had number0 salty cookies and number1 swe...,( 26.0 - 9.0 ),chal-3,"#0: substract ( number0, number3 ) | EOS",26.0 9.0,How many salty cookies did Paco have left?,Subtraction
3,22,number0 children were riding on the bus. At th...,( 43.0 - 21.0 ),chal-4,"#0: substract ( number0, number1 ) | EOS",43.0 21.0,How many children got off the bus at the bus s...,Subtraction
4,2,number0 children were riding on the bus. At th...,( 30.0 - 28.0 ),chal-5,"#0: substract ( number2, number0 ) | EOS",30.0 28.0,How many more children got on the bus than tho...,Subtraction


In [28]:
def token_to_sentence(x):
    y = ""
    for i in x:
        y = y + i + " "
    return y[:-1]
    

In [29]:
body_list = []
question_list = []
linear_formula_list = []
inc_linear_formula_list = []
hint_list = []
number_list = []

nb_inc_list = []

for index, row in SVAMP.iterrows():
    equation = row["Equation"]
    linear_formula = row["Linear_Formula"]
    linear_formula_token = linear_formula.split(" ")
    number = row["Numbers"]
    body_list.append(row["Body"])
    question_list.append(row[question_label])
    linear_formula_list.append(linear_formula)
    inc_linear_formula_list.append(linear_formula)
    hint_list.append("No <hint>")
    number_list.append(number)

    if len(linear_formula_token)>5:
        ope = linear_formula_token[1]
        if ope in ["divide", "substract"]:
            linear_formula_token2 = linear_formula_token.copy()
            a = linear_formula_token2[3]
            linear_formula_token2[3] = linear_formula_token2[4] + ','
            linear_formula_token2[4] = a[0:7]
            hint = 'The number in the position ' + str(3) + " is incorrect."
            hint += ' The number in the position ' + str(4) + " is incorrect. <hint>"
            body_list.append(row["Body"])
            question_list.append(row[question_label])
            linear_formula_list.append(linear_formula)
            inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
            hint_list.append(hint)
            number_list.append(number)
        else:
            linear_formula_token2 = linear_formula_token.copy()
            a = linear_formula_token2[3]
            linear_formula_token2[3] = linear_formula_token2[4] + ','
            linear_formula_token2[4] = a[0:7]
            hint = "No <hint>"
            body_list.append(row["Body"])
            question_list.append(row[question_label])
            linear_formula_list.append(linear_formula)
            inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
            hint_list.append(hint)
            number_list.append(number)
        for other_ope in ["add", "divide", "substract", "multiply"]:
            if other_ope != ope:
                if other_ope in ["divide", "substract"]:
                    # first change ope
                    linear_formula_token2 = linear_formula_token.copy()
                    linear_formula_token2[1] = other_ope
                    hint = 'The operation in the position ' + str(1) + " is incorrect. <hint>"
                    body_list.append(row["Body"])
                    question_list.append(row[question_label])
                    linear_formula_list.append(linear_formula)
                    inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
                    hint_list.append(hint)
                    number_list.append(number)
                    # second change ope and swap nb
                    a = linear_formula_token2[3]
                    linear_formula_token2[3] = linear_formula_token2[4] + ','
                    linear_formula_token2[4] = a[0:7]
                    hint = 'The operation in the position ' + str(1) + " is incorrect."
                    hint += ' The number in the position ' + str(3) + " is incorrect."
                    hint += ' The number in the position ' + str(4) + " is incorrect. <hint>"
                    body_list.append(row["Body"])
                    question_list.append(row[question_label])
                    linear_formula_list.append(linear_formula)
                    inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
                    hint_list.append(hint)
                    number_list.append(number)
                else:
                    # first change ope
                    linear_formula_token2 = linear_formula_token.copy()
                    linear_formula_token2[1] = other_ope
                    hint = 'The operation in the position ' + str(1) + " is incorrect. <hint>"
                    body_list.append(row["Body"])
                    question_list.append(row[question_label])
                    linear_formula_list.append(linear_formula)
                    inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
                    hint_list.append(hint)
                    number_list.append(number)
        # add operations
        if len(linear_formula_token) == 9 or len(linear_formula_token) == 8:
            linear_formula_token2 = linear_formula_token.copy()
            ope_add = random.choice(operation_list2)
            num_add = random.choice(["number0", "number1"])
            if file == "SVAMP":
                linear_formula_token2.remove("EOS")
            linear_formula_token2 = linear_formula_token2 + ["#1"] + [ope_add] + ["("] + ["#0,"] + [num_add] + [")"] + ["|"]
            #0: add ( number0, number1 ) |
            if file == "SVAMP":
                linear_formula_token2.append("EOS")
            hint = "Remove an operation. <hint>"
            body_list.append(row["Body"])
            question_list.append(row[question_label])
            linear_formula_list.append(linear_formula)
            inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
            hint_list.append(hint)
            number_list.append(number)
        # remove operations
        if len(linear_formula_token) > 9:
            linear_formula_token2 = linear_formula_token.copy()
            if file == "SVAMP":
                linear_formula_token2.remove("EOS")
            linear_formula_token2 = linear_formula_token2[:len(linear_formula_token2)-8]
            #0: add ( number0, number1 ) |
            if file == "SVAMP":
                linear_formula_token2.append("EOS")
            hint = "add an operation. <hint>"
            body_list.append(row["Body"])
            question_list.append(row[question_label])
            linear_formula_list.append(linear_formula)
            inc_linear_formula_list.append(token_to_sentence(linear_formula_token2))
            hint_list.append(hint)
            number_list.append(number)
            



In [30]:
incorrect_SVAMP = pd.DataFrame(columns=["Body", "Question", "linear_formula", "inc_linear_formula", "hint", "number"])
incorrect_SVAMP["Body"] = body_list
incorrect_SVAMP["Question"] = question_list
incorrect_SVAMP["linear_formula"] = linear_formula_list
incorrect_SVAMP["inc_linear_formula"] = inc_linear_formula_list
incorrect_SVAMP["hint"] = hint_list
incorrect_SVAMP["number"] = number_list
incorrect_SVAMP

,Body,Question,linear_formula,inc_linear_formula,hint,number
0,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: substract ( number0, number1 ) | EOS",No <hint>,76.0 25.0
1,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: substract ( number1, number0 ) | EOS",The number in the position 3 is incorrect. The...,76.0 25.0
2,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: add ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
3,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
4,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number1, number0 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
...,...,...,...,...,...,...
7344,The grasshopper and the frog had a jumping con...,How far did the frog jump?,"#0: substract ( number0, number1 ) | EOS","#0: add ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,13.0 2.0
7345,The grasshopper and the frog had a jumping con...,How far did the frog jump?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,13.0 2.0
7346,The grasshopper and the frog had a jumping con...,How far did the frog jump?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number1, number0 ) | EOS",The operation in the position 1 is incorrect. ...,13.0 2.0
7347,The grasshopper and the frog had a jumping con...,How far did the frog jump?,"#0: substract ( number0, number1 ) | EOS","#0: multiply ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,13.0 2.0


In [31]:
n = incorrect_SVAMP.shape[0]
m = incorrect_SVAMP.loc[incorrect_SVAMP["hint"]=="No <hint>"].shape[0]
print(m/n)
l = int(n/m-2)
l
x = incorrect_SVAMP.loc[incorrect_SVAMP["hint"]=="No <hint>"]
for i in range(l):
    incorrect_SVAMP = pd.concat([incorrect_SVAMP, x])
incorrect_SVAMP

0.18519526466185876


,Body,Question,linear_formula,inc_linear_formula,hint,number
0,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: substract ( number0, number1 ) | EOS",No <hint>,76.0 25.0
1,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: substract ( number1, number0 ) | EOS",The number in the position 3 is incorrect. The...,76.0 25.0
2,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: add ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
3,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number0, number1 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
4,Each pack of dvds costs number0 dollars. If th...,How much do you have to pay to buy each pack?,"#0: substract ( number0, number1 ) | EOS","#0: divide ( number1, number0 ) | EOS",The operation in the position 1 is incorrect. ...,76.0 25.0
...,...,...,...,...,...,...
7314,Paige was helping her mom plant flowers and to...,How many flower beds did they have?,"#0: divide ( number0, number1 ) | EOS","#0: divide ( number0, number1 ) | EOS",No <hint>,36.0 12.0
7321,"At the zoo, a cage had number0 snakes and numb...",How many alligators were not hiding?,"#0: substract ( number1, number3 ) | EOS","#0: substract ( number1, number3 ) | EOS",No <hint>,75.0 19.0
7328,Paige was helping her mom plant flowers and to...,How many flowers did they grow?,"#0: divide ( number0, number1 ) | #1: multiply...","#0: divide ( number0, number1 ) | #1: multiply...",No <hint>,60.0 55.0 15.0
7335,Mary is baking a cake. The recipe calls for nu...,How many more cups of sugar does she need to add?,"#0: substract ( number0, number2 ) | EOS","#0: substract ( number0, number2 ) | EOS",No <hint>,7.0 4.0


In [32]:
n = incorrect_SVAMP.shape[0]
m = incorrect_SVAMP.loc[incorrect_SVAMP["hint"]=="No <hint>"].shape[0]
m/n

0.4762071378586424

In [33]:
with open(file + "_critic_data.json", 'w') as json_file:
        json.dump(incorrect_SVAMP.to_dict('records'), json_file, indent=0,
                  sort_keys=True)